<a href="https://colab.research.google.com/github/tomast1337/APS-DDD-vs-SOA/blob/main/ITA_DSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install metar@git+https://github.com/python-metar/python-metar.git

  Cloning https://github.com/python-metar/python-metar.git to /tmp/pip-install-wzdbzk18/metar_282e5d9759f7414988c68792730318d9
  Running command git clone --filter=blob:none --quiet https://github.com/python-metar/python-metar.git /tmp/pip-install-wzdbzk18/metar_282e5d9759f7414988c68792730318d9
  Resolved https://github.com/python-metar/python-metar.git to commit a74df97e61f905617f1707641ddddc5ff236050c
  Preparing metadata (setup.py) ... done
  Created wheel for metar: filename=metar-1.11.0-py3-none-any.whl size=201466 sha256=192e3392128ed24d44473745b497d2d8c2c9f7ffab203549b2d9b9bdb36d8449
  Stored in directory: /tmp/pip-ephem-wheel-cache-w7zpceu2/wheels/a2/b3/dc/915c80e5007d00a081b99e51e12495d0f335a78ff1b8ca4ec4
Successfully built metar


In [15]:
# Importa as bibliotecas necessárias
import requests as rq
import json
import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler



from datetime import datetime as dt
from metar import Metar

base = 'http://montreal.icea.decea.mil.br:5002/api/v1/'
token = "a779d04f85c4bf6cfa586d30aaec57c44e9b7173"

In [ ]:
# Realiza uma request
def conectar(dado, data_i='', data_f='', base=base):
  conn = rq.get(f'{base}/{dado}', params={'token':token, 'idate':data_i, 'fdate':data_f})
  return conn

# Checa as informações da conexão
def ver_conn(dados):
  info = json.loads(dados.text)
  pprint.pprint(info)

# Converte os dados para DataFrame
def to_df(dados):
  info = json.loads(dados.text)
  return pd.DataFrame(info)

In [ ]:
conn = conectar('bimtra', '2021-01-01', '2022-11-30')
# Checa a conexão
print(str(conn))

KeyboardInterrupt: ignored

In [13]:
df_test = to_df(conn)
df_test.head(20)


,hora,metar,aero
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
5,1654059600000,METAR SBBR 010500Z 27004KT CAVOK 13/08 Q1018=,SBBR
6,1654063200000,METAR SBBR 010600Z 25003KT CAVOK 14/08 Q1018=,SBBR
7,1654066800000,METAR SBBR 010700Z 28002KT CAVOK 12/08 Q1018=,SBBR
8,1654070400000,METAR SBBR 010800Z VRB01KT CAVOK 11/08 Q1018=,SBBR
9,1654074000000,METAR SBBR 010900Z 26002KT CAVOK 12/09 Q1019=,SBBR


In [ ]:
from pandas._libs.tslibs.parsing import parse_datetime_string
times = []
temps = []
dewpts = []
presses = []
vises = []
wdirs = []
wspds = []
columns = [
    "aero",
    "times",
    "temps",
    "dewpts",
    "presses",
    "vises",
    "wdirs",
    "wspds",
    "colluns",
]

new_df = pd.DataFrame(columns=columns)

index = 0

for i, row in df_test.iterrows():
  METAR = row['metar']
  try:
    parsed = Metar.Metar(METAR)
  except:
    continue
  new_df.loc[index,:] =  [
      index,
      row["aero"],
      parsed.time,
      parsed.temp,
      parsed.dewpt,
      parsed.press,
      parsed.vis,
      parsed.wind_dir,
      parsed.wind_speed
      ]
  index+=1

---

In [29]:
new_df.head(10)

,aero,times,temps,dewpts,presses,vises,wdirs,wspds,colluns
0,0,SBSV,2023-08-30 23:00:00,26.0 C,24.0 C,1008.0 mb,greater than 10000 meters,30 degrees,7 knots


In [ ]:
# Define a funçao que calcula o tempo de vôo com base na partida e chegada
def delta_seg(inicio, fim):
  return (fim - inicio).total_seconds()

In [ ]:
# Mapeia os aeródromos de origem e destino numericamente
aero_map = {'SBBR':0, 'SBCF':1, 'SBCT':2, 'SBFL':3, 'SBGL':4, 'SBGR':5,
            'SBKP':6, 'SBPA':7, 'SBRF':8, 'SBRJ':9, 'SBSP': 10, 'SBSV':11}

df_test['origem_ctg'] = df_test['origem'].map(aero_map)
df_test['destino_ctg'] = df_test['destino'].map(aero_map)


In [ ]:
# Calcula os tempos de vôo (deltas) de cada viagem
deltas = []
for dep, arr in zip(df_test['dt_dep'].values, df_test['dt_arr'].values):
  bgn = dt.fromtimestamp(dep/1000)
  end = dt.fromtimestamp(arr/1000)
  deltas.append(delta_seg(bgn, end))
df_test['delta (s)'] = deltas

In [ ]:
# Função que Normaliza os dados (Opcional)
def normalizar_dados(nomecol_I, nomecol_F):
  scaler = StandardScaler()
  dados = df_test[nomecol_I].values
  reshaped = np.array(dados).reshape(-1, 1)
  scaler.fit(reshaped)
  df_test[nomecol_F] = scaler.transform(reshaped)
  return scaler

In [ ]:
# Normaliza dados (Opcional)
scaler = normalizar_dados('delta (s)', 'T_DELTAS')
print(scaler)

None


In [ ]:
# Visualiza o DF
df_test.head()

# Testa algo

,flightid,origem,destino,dt_dep,dt_arr,origem_ctg,destino_ctg,delta (s),T_DELTAS
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,1654044297000,1654046760000,6,9,2463.0,-0.847046
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,1654045021000,1654047173000,5,9,2152.0,-0.969009
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,1654043212000,1654047825000,0,9,4613.0,-0.003890
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,1654045120000,1654047701000,6,1,2581.0,-0.800770
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,1654045238000,1654050785000,1,3,5547.0,0.362393


In [ ]:
# Segrega as variáveis de previsão e previstas (features e labels)
x = df_test[['origem_ctg', 'destino_ctg']].values
y = df_test['T_DELTAS'].values

# Separa dados de treino e teste
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.8)

# Definição do Modelo
model = Sequential()

model.add(Dense(250, input_dim=2, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='selu'))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mse')

model.fit(x_train, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
866/866 [==============================] - 3s 3ms/step - loss: 0.8213 - val_loss: 0.8222
Epoch 2/100
866/866 [==============================] - 3s 3ms/step - loss: 0.7744 - val_loss: 0.7913
Epoch 3/100
866/866 [==============================] - 3s 3ms/step - loss: 0.7247 - val_loss: 0.8116
Epoch 4/100
866/866 [==============================] - 2s 3ms/step - loss: 0.7199 - val_loss: 0.7359
Epoch 5/100
866/866 [==============================] - 2s 3ms/step - loss: 0.7030 - val_loss: 0.7539
Epoch 6/100
866/866 [==============================] - 3s 3ms/step - loss: 0.6996 - val_loss: 0.7243
Epoch 7/100
866/866 [==============================] - 3s 4ms/step - loss: 0.6931 - val_loss: 0.7185
Epoch 8/100
866/866 [==============================] - 3s 3ms/step - loss: 0.6871 - val_loss: 0.7342
Epoch 9/100
866/866 [==============================] - 3s 3ms/step - loss: 0.6868 - val_loss: 0.7234
Epoch 10/100
866/866 [==============================] - 3s 3ms/step - loss: 0.6845 - val_lo

In [ ]:
model.evaluate(x_test, y_test)

4328/4328 [==============================] - 8s 2ms/step - loss: 0.8647


0.8646800518035889

In [ ]:
def predict_T(mymodel, values, myscaler):
  pred = mymodel.predict([values])
  pred = pred.reshape(-1, 1)
  print(pred)
  pred_T = myscaler.inverse_transform(pred)
  return pred_T

In [ ]:
print(scaler)
predict_T(model, [10, 2], scaler)

None
1/1 [==============================] - 0s 100ms/step
[[-0.82347494]]


AttributeError: ignored